**Construct a regional DSM from xyz point triads covering 1 (or more) global subregions as defined by the U.S.G.S. and Steinwand, 1994 associated with modeling the world (in its entirety) using the Interrupted Goode Homolosine (Sinusoidal component) map projection. **

Construct a DSM of oceanic crustal sediment thicknesses for IGH Region 03 (the S. North America and Central America) using crustal sedient thickness data downloaded from NOAA's NCEI site: https://www.ngdc.noaa.gov/mgg/sedthick/index.html

__Dataset:__ _Total Sediment Thickness of the World's Oceans and Marginal Seas, Version 2_   

__Source Projection__ EPSG::4326

__Reference:__ _Whittaker, Joanne, Alexey Goncharov, Simon Williams, R. Dietmar Müller, German Leitchenkov (2013) Global sediment thickness dataset updated for the Australian-Antarctic Southern Ocean, Geochemistry, Geophysics, Geosystems. DOI: 10.1002/ggge.20181_

__Region 3 geographic extents:__ 40 44'N to 0 180W to 40W Sinusoidal 100W

__Region 3 projection parameters:__  proj +proj=sinu +lon_0=100W +x_0=-11119487.42847 +y_0=0.0Sinusoidal

Sinusoidal Projection

In [ ]:
### using SQLite, locate and extract the data points (triads) for Region 3:

bash# sqlite3 db
> create table tbl(x real, y real, z real);
> .separator " "
> .import sedthick_world_v2.xyz tbl
> .mode csv
> .output sedthick_region03_v2.csv
> select * from tbl where x > 225.0 AND x < 300.0 AND y > 15.0 AND y < 40.733333;
> .quit

In [ ]:
### Project the GCS triads to Sinusoidal meters:

proj +proj=sinu +lon_0=100W +x_0=-11119487.42847 +y_0=0.0 sedthick_region03_v2.gcs >sedthick_region03_v2.sin

In [ ]:
### Import sedthick_region03_v2.csv into GRASS:

## import the point triads:
v.in.ascii -b --overwrite input=/Users/paulp/GoogleDrive/projects/ContinentalMarginMapping/data/sedthick_region03_v2.sin output=SedThkness_xyz separator=space z=3

## create a mask to restrict interpolation region:
v.hull -f --overwrite input=SedThkness_xyz@user output=SedThkness_chull
v.to.rast --overwrite input=SedThkness_chull@user output=SedThkness_msk_1km use=cat

## interpolate to a thickness DSM:
v.surf.rst --overwrite input=SedThkness_xyz@user elevation=SedThickness_1km mask=SedThkness_msk_1km@user npmin=160

## remove the interpolated portion of the DSM that sits over land:
r.mapcalc --o
Crust_Sed_Thkness_1km = if( ETOPO1_bathy_1km, SedThickness_1km, null() )
end

In [ ]:
### Extract an N point random sample from the sediment thickness DSM:

## There are 180507 original data points in Region 03, take an arbitrary 10%: 18,500 random points for further exploration:
r.random --overwrite input=Crust_Sed_Thkness_1km@user npoints=18500 vector=SedThkness_region03_18500randpts_xyz@user

In [ ]:
### Add water depths to the sediment thickness [random] sample data set:

v.what.rast map=SedThkness_region03_18500randpts_xyz@user raster=ETOPO1_bathy_1km@user column=depth

In [ ]:
### export these 18500 random points to an ESRI shape file:
v.out.ogr -e --overwrite input=SedThkness_region03_18500randpts_xyz@user output=/Users/paulp/sedthkness format=ESRI_Shapefile output_layer=SedThkness_region03_18500randpts_xyz